In [16]:

import dotenv

# https://www.promptingguide.ai/zh/techniques/rag
# https://python.langchain.com/docs/use_cases/question_answering/quickstart
# Get the API key from the user 'input' 
# import getpass
# import os
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# dot env ref:
# https://stackoverflow.com/questions/40216311/reading-in-environment-variables-from-an-environment-file

dotenv.load_dotenv("./.env")


True

In [17]:
from langchain_openai import ChatOpenAI

# Models: 
# https://platform.openai.com/docs/models/gpt-3-5-turbo
llm = ChatOpenAI(model="gpt-3.5-turbo")

llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020A088D8490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020A088D9D80>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [18]:
import asyncio
import logging
from typing import Dict, List, Optional, Sequence, Callable

from langchain_core.documents import Document

from langchain_community.document_loaders.base import BaseLoader

logger = logging.getLogger(__name__)


class ExMongodbLoader(BaseLoader):
    """Load MongoDB documents."""

    def __init__(
        self,
        connection_string: str,
        db_name: str,
        collection_name: str,
        *,
        filter_criteria: Optional[Dict] = None,
        field_names: Optional[Sequence[str]] = None,
        doc_transformer: Callable = str,
    ) -> None:
        try:
            from motor.motor_asyncio import AsyncIOMotorClient
        except ImportError as e:
            raise ImportError(
                "Cannot import from motor, please install with `pip install motor`."
            ) from e
        if not connection_string:
            raise ValueError("connection_string must be provided.")

        if not db_name:
            raise ValueError("db_name must be provided.")

        if not collection_name:
            raise ValueError("collection_name must be provided.")

        self.client = AsyncIOMotorClient(connection_string)
        self.db_name = db_name
        self.collection_name = collection_name
        self.filter_criteria = filter_criteria or {}
        self.field_names = field_names or []
        self.doc_transformer = doc_transformer

        self.db = self.client.get_database(db_name)
        self.collection = self.db.get_collection(collection_name)

    def load(self) -> List[Document]:
        """Load data into Document objects.

        Attention:

        This implementation starts an asyncio event loop which
        will only work if running in a sync env. In an async env, it should
        fail since there is already an event loop running.

        This code should be updated to kick off the event loop from a separate
        thread if running within an async context.
        """
        return asyncio.run(self.aload())

    async def aload(self) -> List[Document]:
        """Load data into Document objects."""
        result = []
        total_docs = await self.collection.count_documents(self.filter_criteria)

        # Construct the projection dictionary if field_names are specified
        projection = (
            {field: 1 for field in self.field_names} if self.field_names else None
        )

        async for doc in self.collection.find(self.filter_criteria, projection):
            metadata = {
                "database": self.db_name,
                "collection": self.collection_name,
            }

            # Extract text content from filtered fields or use the entire document
            text = self.doc_transformer(doc)

            result.append(Document(page_content=text, metadata=metadata))

        if len(result) != total_docs:
            logger.warning(
                f"Only partial collection of documents returned. "
                f"Loaded {len(result)} docs, expected {total_docs}."
            )

        return result


In [19]:

# document loaders
# https://python.langchain.com/docs/modules/data_connection/document_loaders/
# https://python.langchain.com/docs/integrations/document_loaders/mongodb

# add this import for running in jupyter notebook
import nest_asyncio

nest_asyncio.apply()

# IT IS TOO WEAK:
# from langchain_community.document_loaders.mongodb import MongodbLoader

def transform_detail_doc(doc: dict) -> str:
    return f'名称：{doc.get("项目名称", "")}; 简介：{doc.get("项目信息", {}).get("项目简介", "")}'


loader = ExMongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="final",
    collection_name="itemDetail",
    doc_transformer=transform_detail_doc,
    field_names="项目信息 项目名称".split(),
)
# KeyError: '项目信息.项目简介'

In [20]:
title_and_abstracts = loader.load()

len(title_and_abstracts)


247965


- 250000 docs
- 20 char/doc
- 5 token/char
- 0.002 rmb/k*token



In [21]:
print(250000*20*5*0.002/1000, "rmb")

50.0 rmb


In [22]:

# 一个文档似乎有点太长了
# 加入简介是否必要？
# 直接切分也不太合适，会出现只有简介的部分，对大模型选题造成干扰

title_and_abstracts[0]


Document(page_content='名称：高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例; 简介：', metadata={'database': 'final', 'collection': 'itemDetail'})

In [23]:


loader = ExMongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="final",
    collection_name="itemDetail",
    field_names= ["项目名称"],
    doc_transformer=lambda x:str(x.get("项目名称", ""))
)
titles = loader.load()
titles = titles[:32]



In [24]:

chars_cnt = 0

for doc in titles:
    text = doc.page_content
    chars_cnt += len(text)

print(chars_cnt/len(titles))


17.96875


In [25]:
[print(x.page_content) for x in titles[:8]]

高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例
大学生对中西方传统节日价值取向的研究
“语伞”雨伞共享平台设计
互联网+大学城闲散物流资源跨界整合创新实践
太阳跟随系统
移动快充跨境电商创业
静态平衡仪
疏勒河流域水-能源-粮食纽带关系研究


[None, None, None, None, None, None, None, None]

In [29]:
from langchain_community.embeddings import QianfanEmbeddingsEndpoint

from langchain_elasticsearch import ElasticsearchStore

# https://python.langchain.com/docs/modules/data_connection/text_embedding/
# https://python.langchain.com/docs/integrations/text_embedding/baidu_qianfan_endpoint/

# https://python.langchain.com/docs/modules/data_connection/vectorstores/
# https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/



embedding = QianfanEmbeddingsEndpoint(model="Embedding-V1")

res = embedding.embed_documents([
    "你好，世界！", 
    "Hello, world!",
    "国内链接OpenAI不太方便，于是我使用百度公司的model。",
])


[INFO] [05-10 19:39:18] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /embeddings/embedding-v1


In [30]:
[print(f"{num:.2f}", end=", ") for num in res[0]]
print()
[print(f"{num:.2f}", end=", ") for num in res[1]]
print()
[print(f"{num:.2f}", end=", ") for num in res[-1]]
type(res[0])

import numpy as np

arrays = [np.array(ls) for ls in res]

print()
print(np.dot(arrays[0], arrays[1]))
print(np.dot(arrays[0], arrays[2]))
print(np.dot(arrays[1], arrays[2]))


-0.02, -0.04, -0.03, 0.10, -0.03, -0.05, 0.01, -0.05, 0.03, 0.01, 0.04, -0.13, 0.08, -0.02, -0.07, -0.02, 0.00, -0.00, 0.03, 0.02, -0.01, -0.04, -0.05, -0.02, 0.03, 0.03, 0.08, 0.05, 0.02, -0.01, -0.00, 0.02, 0.03, -0.06, -0.02, 0.05, 0.04, 0.02, -0.08, 0.04, -0.01, 0.05, -0.00, 0.09, 0.03, -0.06, -0.04, -0.08, -0.08, -0.05, 0.00, 0.01, -0.02, 0.08, -0.01, -0.04, -0.11, -0.02, 0.07, -0.06, 0.01, 0.06, 0.08, -0.06, 0.06, 0.01, -0.00, -0.00, 0.07, 0.01, -0.02, -0.03, 0.06, 0.08, -0.06, 0.04, -0.03, 0.00, -0.00, -0.02, 0.06, -0.08, 0.09, -0.07, -0.08, -0.01, 0.02, 0.04, -0.02, -0.01, -0.01, -0.08, 0.02, -0.08, -0.08, -0.04, 0.02, -0.01, -0.01, -0.07, 0.07, 0.06, 0.07, 0.00, 0.08, -0.05, -0.07, -0.11, 0.01, 0.04, -0.02, 0.05, 0.02, -0.01, -0.05, 0.08, 0.07, -0.02, 0.02, -0.05, -0.07, 0.08, -0.02, 0.11, -0.05, 0.06, -0.12, -0.04, -0.05, 0.06, 0.01, 0.01, 0.07, -0.01, -0.03, 0.00, 0.03, -0.02, -0.04, 0.09, -0.10, -0.02, -0.02, 0.07, 0.05, 0.09, 0.05, -0.01, -0.03, 0.03, -0.03, 0.01, -0.02, 0

In [31]:
elastic_vector_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="rag_test",
    embedding=embedding,
)

elastic_vector_store.client.delete_by_query(query={
    "match_all": {}
}, index="rag_test", ignore=[400, 404],)

elastic_vector_store = ElasticsearchStore.from_documents(
    documents=titles,
    embedding=embedding,
    index_name="rag_test",
    es_url="http://localhost:9200",
)


C:\Users\realj\AppData\Local\Temp\ipykernel_30464\187219858.py:7: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  elastic_vector_store.client.delete_by_query(query={
[INFO] [05-10 19:39:19] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [05-10 19:39:20] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /embeddings/embedding-v1


In [32]:
retriever = elastic_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.invoke("大学生平台")

print(len(retrieved_docs))
[print(doc.page_content) for doc in retrieved_docs]


[INFO] [05-10 19:39:20] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /embeddings/embedding-v1


5
互联网+大学城闲散物流资源跨界整合创新实践
高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例
“海上丝绸之路﹒中国史迹”申报世界文化遗产背景下台山山咀码头、三洲港及沿港临街环境整治设计
大学毕业生面试服装设计与服饰搭配咨询服务工作室
大学生对中西方传统节日价值取向的研究


[None, None, None, None, None]

In [33]:
retrieved_docs = retriever.invoke("创新研究啊")

print(len(retrieved_docs))
[print(doc.page_content) for doc in retrieved_docs]

[INFO] [05-10 19:39:21] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /embeddings/embedding-v1


5
刻纸艺术在室内陈设中的传承与创新
高热导事故容错核燃料芯块的高温高压制备与研究
多旋翼飞行器电网巡检相关问题的研究
盆栽行业新型树屋的造景研发
智能除臭马桶优化设计


[None, None, None, None, None]

In [34]:
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.language_models.chat_models import HumanMessage

# https://python.langchain.com/docs/integrations/chat/
# https://python.langchain.com/docs/integrations/chat/baidu_qianfan_endpoint/

llm = QianfanChatEndpoint(model="ERNIE-3.5-8K", streaming=True)
messages = [HumanMessage(content="你好，我是Jayden")]
llm.invoke(messages)


[WARNING] [05-10 19:39:21] base.py:406 [t:6664]: retry is not available when stream is enabled
[INFO] [05-10 19:39:21] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /chat/completions


AIMessage(content='你好，Jayden！很高兴认识你。请问有什么我可以帮助你的吗？或者我们可以聊聊你感兴趣的话题。', response_metadata={'token_usage': {}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'stop'}, id='run-2cb41953-2cba-473c-bb2f-e98a816f4b0a-0')

In [35]:
from langchain_core.prompts import PromptTemplate

template = """请根据学生情况和往年选题，为学生拟定几个合适的选题。
学生情况：{student_info}
拟定新的选题："""

rag_prompt = PromptTemplate.from_template(template)


In [36]:
ex_stu_info = "信息管理与信息系统专业，自然语言处理方向"

ex_msg = rag_prompt.invoke(
    {
        "student_info": ex_stu_info,
        "history_titles": ";".join([
            doc.page_content for doc in
            retriever.invoke(ex_stu_info, {"k": 5})
        ])
    }
).to_messages()

print(ex_msg[0].content)

[INFO] [05-10 19:39:24] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /embeddings/embedding-v1


请根据学生情况和往年选题，为学生拟定几个合适的选题。
学生情况：信息管理与信息系统专业，自然语言处理方向
拟定新的选题：


In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return ";".join(doc.page_content for doc in docs)


rag_chain = (
    {
        "history_titles": retriever | format_docs, 
        "student_info": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [38]:
for chunk in rag_chain.stream("信息管理与信息系统专业，自然语言处理方向，数学建模比赛经历"):
    print(chunk, end="", flush=True)

[INFO] [05-10 19:39:24] openapi_requestor.py:336 [t:30924]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [05-10 19:39:24] base.py:406 [t:6664]: retry is not available when stream is enabled
[INFO] [05-10 19:39:24] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /chat/completions


基于学生为信息管理与信息系统专业背景，并专注于自然语言处理方向，且有过数学建模比赛的经历，以下是一些合适的选题建议：

1. 基于自然语言处理的信息检索优化模型研究
   选题理由：该选题结合了信息管理与自然语言处理的核心内容，通过数学建模优化信息检索效率，有助于学生利用数学建模技巧解决实际的信息管理问题。

2. 基于深度学习的社交媒体情感分析模型研究
   选题理由：社交媒体数据蕴含着丰富的情感信息，利用深度学习技术对其进行情感分析，有助于理解用户行为、优化信息推荐等。该选题既体现了自然语言处理的应用价值，也能锻炼学生的数学建模能力。

3. 多源信息融合的自然语言处理模型研究
   选题理由：在信息管理领域，多源信息融合是提高信息质量和利用率的重要手段。通过设计自然语言处理模型来实现多源信息的有效融合，有助于提升学生的信息整合和处理能力。

4. 基于自然语言处理的个性化信息推荐系统研究
   选题理由：在信息爆炸的时代，个性化信息推荐成为提高用户体验的关键。通过自然语言处理技术挖掘用户兴趣，结合数学建模优化推荐算法，可以构建出更加精准的个性化信息推荐系统。

5. 基于文本挖掘的企业知识管理系统优化研究
   选题理由：企业知识管理是企业核心竞争力的重要组成部分，通过文本挖掘技术提取和整理企业知识，结合数学建模优化知识管理流程，有助于提高企业的知识利用效率和管理水平。

这些选题都紧密结合了信息管理与信息系统专业的核心知识点，同时突出了自然语言处理方向的应用特点，有助于学生在实践中提升数学建模和问题解决能力。

In [39]:
new_template = """你是大学生创新创业指导中心的专业教授，请你根据学生情况和往年选题，为学生拟定几个合适的选题。
学生情况：{student_info}
往年选题：{history_titles}
注意：严禁重复往年选题内容，**无需解释拟定选题的理由**，一行一个直接列出选题。
拟定新的选题："""

new_rag_prompt = PromptTemplate.from_template(new_template)


In [40]:
new_rag_chain = (
    {
        "history_titles": retriever | format_docs, 
        "student_info": RunnablePassthrough(),
    }
    | new_rag_prompt
    | llm
    | StrOutputParser()
)
for chunk in new_rag_chain.stream("信息管理与信息系统专业，自然语言处理方向，数学建模比赛经历"):
    print(chunk, end="", flush=True)

[INFO] [05-10 19:39:47] openapi_requestor.py:336 [t:26852]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [05-10 19:39:47] base.py:406 [t:6664]: retry is not available when stream is enabled
[INFO] [05-10 19:39:47] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /chat/completions


1. 基于自然语言处理的智能医疗问答系统研究
2. 信息管理与信息系统在智能物流中的应用探索
3. 基于数据挖掘的电商平台用户行为分析与预测
4. 信息管理与信息系统视角下的智慧城市构建策略
5. 基于自然语言处理的在线教育平台学习效果评估系统研发
6. 信息管理与信息系统对供应链管理效率提升的实践研究
7. 面向智慧金融的自然语言处理技术在风险评估中的应用
8. 基于信息管理与信息系统的公共卫生危机应对策略研究
9. 自然语言处理在社交媒体情感分析中的应用与实践
10. 基于数学建模的信息管理系统优化与决策支持研究

In [41]:
for chunk in new_rag_chain.stream("会计学专业，企业税务与决策项目经历，资本结构优化方向"):
    print(chunk, end="", flush=True)

[INFO] [05-10 19:39:56] openapi_requestor.py:336 [t:16080]: requesting llm api endpoint: /embeddings/embedding-v1


[WARNING] [05-10 19:39:56] base.py:406 [t:6664]: retry is not available when stream is enabled
[INFO] [05-10 19:39:56] openapi_requestor.py:336 [t:6664]: requesting llm api endpoint: /chat/completions


1. 企业税务优化与财务决策支持系统研发
2. 基于大数据的税务风险预警与应对策略研究
3. 资本结构优化对企业价值的影响研究
4. 税务筹划对企业现金流及财务绩效的影响分析
5. 基于人工智能的税务审计与风险识别系统研究
6. 企业税务合规性与可持续发展战略的关联研究
7. 税务筹划对企业资本成本及投资决策的影响分析
8. 跨国企业税务优化策略与国际税务合作研究
9. 基于云计算的税务数据处理与可视化平台构建
10. 企业税务筹划中的风险管理及应对策略研究